# Trabalho 03 - Classificação com Árvore de Decisão 

## Importações de bibliotecas

In [75]:
import numpy as np
import pandas as pd

from typing import Tuple

## Preparando a base de dados

Primeiro irei carregar a base de dados do csv e depois vou transformar as classes "good" e "vgood" na classe "acc" para que o problema se torne binário. Basicamente vou subistituir cada instância de vgoog e good por acc.

In [76]:
car_df = pd.read_csv("car.csv")

car_df["Car_Acceptability"] = car_df["Car_Acceptability"].replace(
    {"good": "acc", "vgood": "acc"}
)

car_df.tail(10)


,Buying_Price,Maintenance_Price,No_of_Doors,Person_Capacity,Size_of_Luggage,Safety,Car_Acceptability
1718,low,low,5more,4,big,high,acc
1719,low,low,5more,more,small,low,unacc
1720,low,low,5more,more,small,med,acc
1721,low,low,5more,more,small,high,acc
1722,low,low,5more,more,med,low,unacc
1723,low,low,5more,more,med,med,acc
1724,low,low,5more,more,med,high,acc
1725,low,low,5more,more,big,low,unacc
1726,low,low,5more,more,big,med,acc
1727,low,low,5more,more,big,high,acc


## Criando uma função para medir entropia


In [77]:
def calcEntropy(labels):
    """Calcula a entropia de uma série de rótulos"""

    valores, contagens = np.unique(labels, return_counts=True)
    probabilidades = contagens / len(labels)

    return -np.sum([p * np.log2(p) for p in probabilidades if p > 0])

## Criando a árvore


In [78]:
class decisionTreeNode:

    def __init__(self, depth: int = 0, max_depth: int = 4):
        
        self.attribute = None
        self.nodes = {}
        self.depth = depth
        self.max_depth = max_depth
        self.predicted_class = None
        self.information_gain = 0 

    
    def fit(self, X, y, available_atributes):

        if len(set(y)) == 1:
            self.predicted_class = y.iloc[0]
            return
        
        # Condições de parada: Todos os atributos utilizados ou profundidade máxima atingida
        if len(available_atributes) == 0 or self.depth >= self.max_depth:
            self.predicted_class = y.mode()[0]
            return
        
        base_entropy = calcEntropy(y)

        best_attribute = None
        best_gain = -1

        # Calculando o ganho de informação de cada atributo
        for attribute in available_atributes:
            gain = self._information_gain(X,y, attribute, base_entropy)

            if gain > best_gain:
                best_gain = gain
                best_attribute = attribute
        
        # Condição de parada: Sem ganho de informação
        # Também salvo o ganho de informação no nó
        if best_gain <= 0:
            self.predicted_class = y.mode()[0]
            self.information_gain = best_gain 
            return
        
        self.attribute = best_attribute
        self.information_gain = best_gain
        
        remaning_attributes = [a for a in available_atributes if a !=best_attribute]

        # Criando nós filhos 
        for value in X[best_attribute].unique():
            subset_X = X[X[best_attribute] == value]
            subset_y = y[X[best_attribute] == value]

            child_node = decisionTreeNode(depth= self.depth+1, max_depth= self.max_depth)
            child_node.fit(subset_X, subset_y, remaning_attributes)

            self.nodes[value] = child_node
        
    
    def _information_gain(self, X, y, attribute, base_entropy) -> float:

        # Pego todos os possíveis valores para um dado tipo de atributo
        values = X[attribute].unique()
        weighted_entropy = 0


        for value in values:
            """
            1. Em cada valor obtido acima, filtro amostras em y que tenham o 
            mesmo valor no atributo.
            2. Pondero o peso dividindo a quantidade de amostas com aquele valor
            pela quantidade total de amostras.
            3. Calulo a entropia daquele subconjunto e depois calculo a entropia 
            ponderada.
            4. O ganho de entropia é a entropia atual menos a entropia ponderada.
            """
            subset_y = y[X[attribute] == value]
            weight = len(subset_y) / len(y)

            entropy_subset = calcEntropy(subset_y)
            weighted_entropy += weight * entropy_subset
        
        information_gain = base_entropy - weighted_entropy

        return information_gain

    def predict(self, sample):
        """
        Essa função percorre a árvore até encontrar uma folha com a classe prevista
        """

        # Retorna a classe prevista
        if self.predicted_class is not None:
            return self.predicted_class
        
        attr_value = sample[self.attribute]
        
        # Se o valor estiver entre os nós filhos, continua descendo
        if attr_value in self.nodes:
            return self.nodes[attr_value].predict(sample)
        
        # Se o valor for desconhecido, retorna a classe mais comum
        return self.predicted_class

    def total_information_gain(self):
        if not self.nodes:  # Se nó folha
            return 0
        total_gain = self.information_gain
        for child in self.nodes.values():
            total_gain += child.total_information_gain()
        return total_gain
    

## Criando um modelo de K-Fold

In [79]:
class KFold:
    """Classificador K-Fold de validação cruzada"""

    def __init__(self, n_splits: int = 10, shuffle: bool = True, random_seed: int = None):
        self.n_splits = n_splits
        self.shuffle = shuffle
        self.random_seed = random_seed

    def split(self, features: np.array, labels=np.array)-> Tuple[np.ndarray, np.ndarray]:
        
        """"Divide os dados em n partes para validação cruzada
            > [INPUT]: Array de features e labels\n
            > [OUTPUT]: Tupla de arrays com os índices de treino e teste para cada fold
        """

        n_samples = len(features)
        indices = np.arange(n_samples)

        if self.shuffle:
            rng = np.random.default_rng(self.random_seed)
            indices = rng.permutation(indices)
        
        fold_sizes = np.full(self.n_splits, n_samples // self.n_splits, dtype=int)
        fold_sizes[:n_samples % self.n_splits] += 1  # Distribui o resto


        current = 0
        for fold_size in fold_sizes:
            start, stop = current, current + fold_size

            #indices para teste e treino
            test_idx = indices[start:stop]
            train_idx = np.concatenate([indices[:start], indices[stop:]])

            current = stop
            yield train_idx, test_idx

## Funções auxiliares para as saídas do algorítimo
- ganho de informação
- acurácia
- sensibilidade
- especificidade
- precisão
- f1-score


In [80]:
def accuracy(y_true, y_pred):
    return np.sum(y_true == y_pred) / len(y_true)

def confusion_matrix(y_true, y_pred, positive_label):
    tp = np.sum((y_true == positive_label) & (y_pred == positive_label))
    tn = np.sum((y_true != positive_label) & (y_pred != positive_label))
    fp = np.sum((y_true != positive_label) & (y_pred == positive_label))
    fn = np.sum((y_true == positive_label) & (y_pred != positive_label))
    return tp, tn, fp, fn

def precision(tp, fp):
    return tp / (tp + fp) if (tp + fp) > 0 else 0

def recall(tp, fn):
    return tp / (tp + fn) if (tp + fn) > 0 else 0

def f1_score(p, r):
    return 2 * p * r / (p + r) if (p + r) > 0 else 0

def specificity(tn, fp):
    return tn / (tn + fp) if (tn + fp) > 0 else 0

def cross_validate(X, y, kfold, max_depth=4):

    gains = []
    accs = []
    senss = []
    specs = []
    precs = []
    f1s = []

    positive_label = y.mode()[0]

    for train_idx, test_idx in kfold.split(X.values, y.values):
        X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
        y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]

        atributos = list(X.columns)
        tree = decisionTreeNode(max_depth=max_depth)
        tree.fit(X_train, y_train, atributos)

        # Soma do ganho de informação total da árvore treinada neste fold
        gains.append(tree.total_information_gain())

        y_pred = []
        for i in range(len(X_test)):
            sample = X_test.iloc[i]
            pred = tree.predict(sample)
            y_pred.append(pred)
        y_pred = np.array(y_pred)

        acc = accuracy(y_test.values, y_pred)
        tp, tn, fp, fn = confusion_matrix(y_test.values, y_pred, positive_label)

        p = precision(tp, fp)
        r = recall(tp, fn)
        f1 = f1_score(p, r)
        spec = specificity(tn, fp)

        accs.append(acc)
        senss.append(r)
        specs.append(spec)
        precs.append(p)
        f1s.append(f1)

    print(f'Média Ganho de Informação: {np.mean(gains):.4f}')
    print(f'Média Acurácia: {np.mean(accs):.4f}')
    print(f'Média Sensibilidade: {np.mean(senss):.4f}')
    print(f'Média Especificidade: {np.mean(specs):.4f}')
    print(f'Média Precisão: {np.mean(precs):.4f}')
    print(f'Média F1-Score: {np.mean(f1s):.4f}')

## Realizando o treino com k-fold = 10 e árvore de decisão

In [81]:
# Separarando atributos e rótulos
X = car_df.drop(columns=['Car_Acceptability'])
y = car_df['Car_Acceptability']

for col in X.columns:
    X[col] = X[col].astype('category').cat.codes
y = y.astype('category')

kfold = KFold(n_splits=10, shuffle=True, random_seed=42)
cross_validate(X, y, kfold, max_depth=4)

Média Ganho de Informação: 7.3756
Média Acurácia: 0.9265
Média Sensibilidade: 0.9369
Média Especificidade: 0.9028
Média Precisão: 0.9578
Média F1-Score: 0.9470
